# 🚀 NLP Project: Benchmark 3 Models (SVM - LSTM - PhoBERT)

Notebook này được tối ưu hóa cho **Google Colab T4 GPU**.

**Quy trình:**
1.  **Setup**: Cài đặt thư viện & Mount Google Drive.
2.  **Data**: Load dữ liệu từ folder `final` trên Drive.
3.  **Train & Eval**:
    * 🤖 **SVM**: Machine Learning cơ bản (Baseline).
    * 🧠 **LSTM**: Deep Learning (PyTorch).
    * 🔥 **PhoBERT**: Transformer (State-of-the-Art).
4.  **Report**: Xuất kết quả ra file Excel và biểu đồ.

In [1]:
# --- 1. SETUP & CONFIG ---
!pip install -q pyvi transformers torch scikit-learn openpyxl matplotlib seaborn

import os
import torch
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path
from google.colab import drive
from pyvi import ViTokenizer
import gc

# Mount Google Drive
drive.mount('/content/drive')

# Kiểm tra GPU
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"\n✅ Đang chạy trên thiết bị: {device}")
if device.type == 'cuda':
    print(f"   ➤ GPU Name: {torch.cuda.get_device_name(0)}")
    print(f"   ➤ Memory: {torch.cuda.get_device_properties(0).total_memory / 1e9:.2f} GB")
else:
    print("❌ CẢNH BÁO: Không tìm thấy GPU! Hãy vào Runtime -> Change runtime type -> T4 GPU.")

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.5/8.5 MB 50.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 56.5 MB/s eta 0:00:00
Mounted at /content/drive

✅ Đang chạy trên thiết bị: cuda
   ➤ GPU Name: Tesla T4
   ➤ Memory: 15.83 GB


In [2]:
# --- 2. LOAD DATA ---
# 👇 SỬA ĐƯỜNG DẪN NÀY NẾU CẦN 👇
DRIVE_PATH = Path("/content/drive/MyDrive/final")
JSONL_PATH = Path("/content/drive/MyDrive/final/nlp_dataset.jsonl")

print("⏳ Đang chuẩn bị dữ liệu...")

# Nếu chưa có file jsonl thì tạo mới từ folder raw
# 1. Đọc file jsonl
if JSONL_PATH.exists():
    print(f"   Đọc từ file cache: {JSONL_PATH}")
    df = pd.read_json(JSONL_PATH, lines=True)
else:
    # Nếu chưa có thì quét từ Drive (Backup)
    print("   Không thấy file cache, đang quét từ Drive...")
    data = []
    if DRIVE_PATH.exists():
        for folder in DRIVE_PATH.iterdir():
            if folder.is_dir():
                topic = folder.name
                files = list(folder.glob("**/*.txt"))[:2000] # Lấy mẫu 2000 bài
                for file_path in files:
                    try:
                        with open(file_path, "r", encoding="utf-16") as f: content = f.read().strip()
                    except:
                        try:
                            with open(file_path, "r", encoding="utf-8") as f: content = f.read().strip()
                        except: continue
                    if content:
                        data.append({
                            "text": ViTokenizer.tokenize(content),
                            "raw_text": content,
                            "label_name": topic, # <--- Cột này là label_name
                            "filename": file_path.name
                        })
        df = pd.DataFrame(data)
        df.to_json(JSONL_PATH, orient="records", lines=True)
    else:
        print("❌ LỖI: Không tìm thấy folder data!")

print(f"📊 Tổng số bài: {len(df)}")
print(f"👀 Các cột hiện có: {list(df.columns)}")

# --- QUAN TRỌNG: CHỌN ĐÚNG CỘT NHÃN ---
# Dữ liệu của bạn dùng 'label_name', nên ta trỏ thẳng vào nó
target_col = 'label_name'

print(f"✅ Đang dùng cột nhãn: {target_col}")

# Mã hóa nhãn (Label Encoding)
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split

le = LabelEncoder()
df['label_id'] = le.fit_transform(df[target_col])
num_classes = len(le.classes_)

# Chia tập Train/Test
# stratify=df[target_col] giúp chia đều tỉ lệ các nhãn
train_df, test_df = train_test_split(df, test_size=0.2, random_state=42, stratify=df[target_col])

print(f"   Train: {len(train_df)} | Test: {len(test_df)}")
print(f"   Số lượng nhãn: {num_classes}")

# Dọn dẹp RAM
del df
gc.collect()

⏳ Đang chuẩn bị dữ liệu...
   Đọc từ file cache: /content/drive/MyDrive/final/nlp_dataset.jsonl
📊 Tổng số bài: 115188
👀 Các cột hiện có: ['text', 'label_name', 'label_id', 'filename']
✅ Đang dùng cột nhãn: label_name
   Train: 92150 | Test: 23038
   Số lượng nhãn: 20


0

## 🤖 Model 1: SVM (Machine Learning)

In [ ]:
print("⏳ Đang huấn luyện SVM...")
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.svm import LinearSVC
from sklearn.metrics import accuracy_score, classification_report

# Feature Engineering: TF-IDF
tfidf = TfidfVectorizer(max_features=20000, ngram_range=(1, 2))
X_train_tfidf = tfidf.fit_transform(train_df['text'])
X_test_tfidf = tfidf.transform(test_df['text'])

# Training
svm = LinearSVC(dual=False)
svm.fit(X_train_tfidf, train_df['label_id'])

# Evaluation
y_pred_svm = svm.predict(X_test_tfidf)
acc_svm = accuracy_score(test_df['label_id'], y_pred_svm)
print(f"✅ SVM Accuracy: {acc_svm:.4f}")

⏳ Đang huấn luyện SVM...


## 🧠 Model 2: LSTM (PyTorch)

In [ ]:
print("⏳ Đang chuẩn bị dữ liệu cho LSTM...")
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from collections import Counter

# 1. Xây bộ từ điển (Vocab)
def build_vocab(texts, max_features=20000):
    counter = Counter()
    for text in texts: counter.update(text.split())
    vocab = {word: i+2 for i, (word, _) in enumerate(counter.most_common(max_features))}
    vocab['<PAD>'] = 0; vocab['<UNK>'] = 1
    return vocab

vocab = build_vocab(train_df['text'])
MAX_LEN = 200

# 2. Hàm chuyển text sang số
def text_to_seq(text, vocab, max_len):
    seq = [vocab.get(word, 1) for word in text.split()]
    if len(seq) < max_len: seq += [0] * (max_len - len(seq))
    else: seq = seq[:max_len]
    return seq

# 3. Dataset Class
class LSTMDataset(Dataset):
    def __init__(self, df, vocab):
        self.x = [text_to_seq(t, vocab, MAX_LEN) for t in df['text']]
        self.y = df['label_id'].values
    def __len__(self): return len(self.y)
    def __getitem__(self, idx):
        return torch.tensor(self.x[idx], dtype=torch.long), torch.tensor(self.y[idx], dtype=torch.long)

train_ds_lstm = LSTMDataset(train_df, vocab)
test_ds_lstm = LSTMDataset(test_df, vocab)
train_loader_lstm = DataLoader(train_ds_lstm, batch_size=64, shuffle=True)
test_loader_lstm = DataLoader(test_ds_lstm, batch_size=64)

# 4. Model Architecture
class LSTMClassifier(nn.Module):
    def __init__(self, vocab_size, embed_dim, hidden_dim, n_classes):
        super().__init__()
        self.embedding = nn.Embedding(vocab_size, embed_dim)
        self.lstm = nn.LSTM(embed_dim, hidden_dim, batch_first=True, dropout=0.2)
        self.fc = nn.Linear(hidden_dim, n_classes)
    def forward(self, x):
        x = self.embedding(x)
        _, (hn, _) = self.lstm(x)
        return self.fc(hn[-1])

model_lstm = LSTMClassifier(len(vocab)+2, 100, 100, num_classes).to(device)
optimizer = torch.optim.Adam(model_lstm.parameters(), lr=0.001)
criterion = nn.CrossEntropyLoss()

# 5. Training Loop
print("🚀 Bắt đầu Train LSTM...")
for epoch in range(5):
    model_lstm.train()
    for x, y in train_loader_lstm:
        x, y = x.to(device), y.to(device)
        optimizer.zero_grad()
        loss = criterion(model_lstm(x), y)
        loss.backward()
        optimizer.step()

# 6. Evaluate
model_lstm.eval()
preds, targets = [], []
with torch.no_grad():
    for x, y in test_loader_lstm:
        output = model_lstm(x.to(device))
        preds.extend(torch.argmax(output, dim=1).cpu().numpy())
        targets.extend(y.numpy())
acc_lstm = accuracy_score(targets, preds)
print(f"✅ LSTM Accuracy: {acc_lstm:.4f}")

## 🔥 Model 3: PhoBERT (Transformer - PyTorch)

In [ ]:
print("⏳ Đang load PhoBERT...")
from transformers import AutoTokenizer, AutoModelForSequenceClassification, AdamW

tokenizer = AutoTokenizer.from_pretrained("vinai/phobert-base")

class PhoBERTDataset(Dataset):
    def __init__(self, df, tokenizer, max_len=128):
        self.texts = df['raw_text'].tolist()
        self.labels = df['label_id'].tolist()
        self.tokenizer = tokenizer
        self.max_len = max_len
    def __len__(self): return len(self.labels)
    def __getitem__(self, idx):
        enc = self.tokenizer(self.texts[idx], truncation=True, padding='max_length', max_length=self.max_len, return_tensors='pt')
        return {
            'input_ids': enc['input_ids'].squeeze(),
            'attention_mask': enc['attention_mask'].squeeze(),
            'labels': torch.tensor(self.labels[idx], dtype=torch.long)
        }

# Lấy mẫu nhỏ hơn để Train PhoBERT cho nhanh (nếu máy yếu có thể giảm xuống 5000)
train_ds_bert = PhoBERTDataset(train_df[:5000], tokenizer)
test_ds_bert = PhoBERTDataset(test_df[:1000], tokenizer)
train_loader_bert = DataLoader(train_ds_bert, batch_size=16, shuffle=True)
test_loader_bert = DataLoader(test_ds_bert, batch_size=16)

model_bert = AutoModelForSequenceClassification.from_pretrained("vinai/phobert-base", num_labels=num_classes)
model_bert.to(device)
optimizer = AdamW(model_bert.parameters(), lr=2e-5)

print("🚀 Bắt đầu Fine-tune PhoBERT (Khoảng 5-10 phút)...")
for epoch in range(2):
    model_bert.train()
    total_loss = 0
    for batch in train_loader_bert:
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        labels = batch['labels'].to(device)

        optimizer.zero_grad()
        outputs = model_bert(input_ids, attention_mask=attention_mask, labels=labels)
        loss = outputs.loss
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
    print(f"   Epoch {epoch+1} Loss: {total_loss/len(train_loader_bert):.4f}")

# Evaluate
model_bert.eval()
preds_bert, targets_bert = [], []
with torch.no_grad():
    for batch in test_loader_bert:
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        outputs = model_bert(input_ids, attention_mask=attention_mask)
        preds_bert.extend(torch.argmax(outputs.logits, dim=1).cpu().numpy())
        targets_bert.extend(batch['labels'].numpy())

acc_bert = accuracy_score(targets_bert, preds_bert)
print(f"✅ PhoBERT Accuracy: {acc_bert:.4f}")

In [ ]:
# --- 4. TỔNG KẾT & XUẤT BÁO CÁO ---
results_df = pd.DataFrame([
    {"Model": "SVM (Baseline)", "Accuracy": acc_svm},
    {"Model": "LSTM (PyTorch)", "Accuracy": acc_lstm},
    {"Model": "PhoBERT (Transformer)", "Accuracy": acc_bert}
]).sort_values(by="Accuracy", ascending=False)

print("\n🏆 BẢNG XẾP HẠNG:")
display(results_df)

# Lưu vào Drive
REPORT_DIR = DRIVE_PATH.parent / "reports"
REPORT_DIR.mkdir(parents=True, exist_ok=True)
save_path = REPORT_DIR / "final_benchmark_t4.xlsx"
results_df.to_excel(save_path, index=False)
print(f"\n💾 Đã lưu file Excel tại: {save_path}")

# Vẽ biểu đồ
plt.figure(figsize=(10, 6))
sns.barplot(x="Accuracy", y="Model", data=results_df, palette="viridis")
plt.xlim(0.0, 1.0)
plt.title("So sánh độ chính xác trên T4 GPU")
plt.savefig(REPORT_DIR / "chart_benchmark.png")
plt.show()